In [1]:
import pandas as pd 
import numpy as np

In [3]:
appl = pd.read_csv('/content/drive/MyDrive/appl.csv')
balance = pd.read_csv('/content/drive/MyDrive/balance.csv')

In [4]:
Status_mapping = {"C": -1, "X": -2, "0": 0, "1":1, "2":2, "3":3, "4":4, "5":5}
balance['STATUS'] = balance['STATUS'].map(Status_mapping)
balance['STATUS'].value_counts()

-1    442031
 0    383120
-2    209230
 1     11090
 5      1693
 2       868
 3       320
 4       223
Name: STATUS, dtype: int64

#### Агрегируем данные для каждого клиента 

In [5]:
status_agg = balance[['ID','STATUS']].groupby('ID').agg(max)
display(status_agg['STATUS'].value_counts())
# display(defaults_1['STATUS'].count())

 0    34682
 1     4683
-2     4536
-1     1417
 2      336
 5      195
 3       88
 4       48
Name: STATUS, dtype: int64

#### Удалим всех тех, у кого нет кредитной истории

In [6]:
status_agg = status_agg.drop(status_agg[status_agg['STATUS'] < -1].index)
print(status_agg)

         STATUS
ID             
5001711       0
5001712       0
5001717       0
5001718       1
5001719       0
...         ...
5150480       0
5150482       0
5150484       0
5150485       0
5150487      -1

[41449 rows x 1 columns]


In [7]:
status_agg['STATUS'].replace({-1: 0, 0:0, 1:0, 2:1, 3:1, 4:1, 5:1}, inplace=True)
status_agg['STATUS'] = status_agg['STATUS'].astype('int')

#### Объединение датасета

In [8]:
df = appl.join(status_agg, on='ID', how='inner').set_index("ID")

#### Проверим на наличие дубликатов и удалим их

In [9]:
len(df)-len(df.drop_duplicates())

23531

In [10]:
df = df.drop_duplicates()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9579 entries, 5008804 to 5150337
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   CODE_GENDER          9579 non-null   object 
 1   FLAG_OWN_CAR         9579 non-null   object 
 2   FLAG_OWN_REALTY      9579 non-null   object 
 3   CNT_CHILDREN         9579 non-null   int64  
 4   AMT_INCOME_TOTAL     9579 non-null   float64
 5   NAME_INCOME_TYPE     9579 non-null   object 
 6   NAME_EDUCATION_TYPE  9579 non-null   object 
 7   NAME_FAMILY_STATUS   9579 non-null   object 
 8   NAME_HOUSING_TYPE    9579 non-null   object 
 9   DAYS_BIRTH           9579 non-null   int64  
 10  DAYS_EMPLOYED        9579 non-null   int64  
 11  FLAG_MOBIL           9579 non-null   int64  
 12  FLAG_WORK_PHONE      9579 non-null   int64  
 13  FLAG_PHONE           9579 non-null   int64  
 14  FLAG_EMAIL           9579 non-null   int64  
 15  OCCUPATION_TYPE      6614 non

In [12]:
# Получили очень несбаланированную выборку
df['STATUS'].value_counts()

0    9137
1     442
Name: STATUS, dtype: int64

In [ ]:
df.to_csv('abc.csv')